# Solution
## Step 1: Create dictionary
- Load the vocab file into numpy using readlines
## Step 2: Create paragraph vectors
- Load all the trainign and test data into an array. This will create 4x12500 length arrays.
 - train/pos
 - train/neg
 - test/pos
 - test/neg
 each of these arays in fact includes a paragraph
## Step 3: Cteate BOW
- tokenize each element in each array. This shanges shape of the vectors. Initially vector shapes are (12500, 1). 
- do wordcount (word, count)
- get rid of most common words
- 


In [5]:
# imports and env-variable
# imports and env-variable
import mxnet as mx
import numpy as np
import os
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
import urllib3
import tarfile
from nltk.corpus import stopwords
from mxnet import nd

BASE_DIR = '/home/ubuntu/CyrusProjects/mxnet-notebooks/python/tutorials'
#BASE_DIR = os.getcwd()
PATH='/aclImdb'

In [2]:
os.chdir('/home/ubuntu/CyrusProjects/mxnet-notebooks/python/tutorials/')
os.system('pip3 install nltk')
nltk.download('all')




[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/ubuntu/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

[nltk_data]    |   Package state_union is already up-to-date!
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package stopwords is already up-to-date!
[nltk_data]    | Downloading package subjectivity to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package subjectivity is already up-to-date!
[nltk_data]    | Downloading package swadesh to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package swadesh is already up-to-date!
[nltk_data]    | Downloading package switchboard to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package switchboard is already up-to-date!
[nltk_data]    | Downloading package timit to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     /home/ubuntu/nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[n

True

In [5]:
http = urllib3.PoolManager()
url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
path = os.getcwd() + '/local_aclImdb_v1.tar.gz'
chunk_size = 1024

r = http.request('GET', url, preload_content=False)

with open(path, 'wb') as out:
    while True:
        data = r.read(chunk_size)
        if not data:
            break
        out.write(data)

r.release_conn()

In [86]:
tar = tarfile.open("aclImdb_v1.tar.gz")
tar.extractall()
tar.close()

In [23]:
with open('aclImdb/imdb.vocab') as vocab_file:
    vocab_array = vocab_file.readlines()
vocab_array = [x.strip() for x in vocab_array]
subset = vocab_array[0:20]
print(len(vocab_array))



89527


In [4]:
with open(BASE_DIR + PATH + '/imdb.vocab') as f:
    dic = f.readlines()
dic= [x.strip() for x in dic]
dic = np.array(np.sort(dic))
dic = np.unique(dic)
dic = dic.tolist()
filtered_dic = [word for word in dic if word not in stopwords.words('english')]
print(len(dic))
print(len(filtered_dic))

89527
89380


In [5]:
def sentence_array_creator(path, root_dir):
    os.chdir(path)
    sentences = []
    for file in list(glob.glob("*.txt")):
        with open(file, 'r') as f:
            sentences.append(f.readline().strip().lower())
    return sentences
    os.chdir(root_dir)



In [6]:
os.chdir(BASE_DIR)
root_dir = os.getcwd()

base_dir = BASE_DIR + PATH
pos_train_path = base_dir + '/train/pos'
neg_train_path = base_dir + '/train/neg'
pos_test_path = base_dir + '/test/pos'
neg_test_path = base_dir + '/test/neg'


pos_train_sentences = sentence_array_creator(pos_train_path, root_dir)
neg_train_sentences = sentence_array_creator(neg_train_path, root_dir)
pos_test_sentences = sentence_array_creator(pos_test_path, root_dir)
neg_test_sentences = sentence_array_creator(neg_test_path, root_dir)
print('{}; {}; {}; {}; '.format)

print("{}, {}, {}, {}".format(len(pos_train_sentences), len(neg_train_sentences), len(pos_test_sentences), len(neg_test_sentences)))



<built-in method format of str object at 0x7f7411be26a8>
12500, 12500, 12500, 12500


In [8]:
pos_train_sentences[0:2]

['in the seemingly endless quest to find well made, well acted horror films, it is all-too-rare to find one that even comes remotely close to hitting the mark. needless to say, i was very pleasantly surprised when i stumbled across "burned at the stake" on a u.s. cable network while i was flipping channels. the premise is reasonably simple. in 1692, young ann putnam (swift) is the most vocal witness against alleged witches, leveling baseless charges against anyone who earns her displeasure. manipulating her for his own ends is reverend parris (peters) who also serves as the court\'s guide on matters pertaining to witchcraft and satanism. things get complicated when ann starts accusing members of the goode family of witchcraft. salem (of 1980 or so), loreen graham (also played by swift) begins having unusual visions shortly before she visits the salem witch museum. a strange man in seventeenth century garb tries to accost her there and the building. he continues to stalk her while stran

In [9]:
def get_dic_index(word, dic):
    try:
        ret_val = dic.index(word)
    except ValueError: 
        ret_val = -1
    return ret_val

    
word = "is"
print(get_dic_index(word, dic))
    

39813


In [10]:
def sentence_encoder(sentences, vocab):
    bow = []
    t = RegexpTokenizer(r'\w+')
    prog = 0
    for sentence in sentences:
        if prog % 10 == 0:
            print(prog, sep=' ', end='-',flush=True)
        s = ''.join(sentence)
        tokens = t.tokenize(s)
        indexes = []
        for token in tokens:
            idx = get_dic_index(token, vocab)
            if idx != -1:
                indexes.append(idx)
        prog += 1
        bow.append(indexes)
    return bow

def sentence_tokenizer(sentences):
    bow = []
    t = RegexpTokenizer(r'\w+')
    for sentence in sentences:
        s = ''.join(sentence)
        tokens = t.tokenize(s)
        bow.append(tokens)
    return bow


In [11]:
def print_items(array):
    for i in array:
        print('{} \n'.format(i))
        




In [ ]:
from mxnet import nd
nd.a

In [24]:
print("now calculating bow_pos_train_sentences...")
bow_pos_train_sentences = sentence_encoder(pos_train_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_pos_train_sentences_batch', bow_pos_train_sentences)

print("now calculating bow_neg_train_sentences...")
bow_neg_train_sentences = sentence_encoder(neg_train_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_neg_train_sentences_batch', bow_neg_train_sentences)

print("now calculating bow_pos_test_sentences...")
bow_pos_test_sentences = sentence_encoder(pos_test_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_pos_test_sentences_batch', bow_pos_test_sentences)

print("now calculating bow_neg_test_sentences...")
bow_neg_test_sentences = sentence_encoder(neg_test_sentences, filtered_dic)
np.save(BASE_DIR + PATH + '/bow_neg_test_sentences_batch', bow_neg_test_sentences)

print("DONE...")

print(len(bow_neg_test_sentences))
print(len(bow_neg_train_sentences))
print(len(bow_pos_test_sentences))
print(len(bow_pos_train_sentences))


now calculating bow_pos_train_sentences...


NameError: name 'sentence_encoder' is not defined

In [75]:
x_pos = np.load(BASE_DIR + PATH + '/bow_pos_train_sentences_batch.npy')
x_neg = np.load(BASE_DIR + PATH + '/bow_neg_train_sentences_batch.npy')
y_pos = np.load(BASE_DIR + PATH + '/bow_pos_test_sentences_batch.npy')
y_neg = np.load(BASE_DIR + PATH + '/bow_neg_test_sentences_batch.npy')

print('{}, {}, {}, {}'.format(len(x_pos), len(x_neg), len(y_pos), len(y_neg)))
print(x_pos[0])
print(x_neg[0])
print(y_pos[0])
print(y_neg[0])

12500, 12500, 12500, 12500
[69223, 33526, 34283, 27714, 42043, 35732, 49802, 4640, 27614, 25684, 67113, 27033, 32642, 711, 69871, 34283, 35074, 28680, 28555, 79695, 35074, 25480, 87221, 59295, 58065, 2187, 86460, 57208, 60105, 58343, 13366, 31982, 35074, 83551, 8694, 70250, 18567, 39239, 27033, 24306, 54662, 18566, 39287, 73946, 25123, 27614, 30970, 7630, 8430, 69079, 33538, 26595, 33526, 34283, 57412, 55150, 49811, 53994, 34283, 27907, 79763, 34643, 27614, 46606, 78284, 49146, 85879, 12673, 23063, 27714, 77736, 55150, 25017, 69225, 35074, 88652, 887, 10925, 87920, 79766, 64932, 12182, 23068, 46778, 63512, 58640]
[59414, 12183, 69218, 39222, 24470, 63875, 44432, 9895, 51434, 81723, 81454, 8858, 50955, 86474, 86092, 87790, 27769, 9124, 9124, 34643, 87790, 51450, 27614, 64801, 29052, 39222, 81428, 75383, 30835, 67464, 57976, 83159, 89102, 12444, 19916, 52645, 30970, 12341, 82921, 21642, 30970, 31876, 69777, 39320, 12482, 56626, 20516, 78313, 43727, 26151, 8858, 50955, 9124, 9124, 31346, 

In [76]:
x_label_pos = np.ones(len(x_pos))
x_label_neg = np.zeros(len(x_neg))
y_label_pos = np.ones(len(y_pos))
y_label_neg = np.zeros(len(y_neg))
print(x_label_pos)
print(y_label_neg)


seed = 113

x_data = np.concatenate([x_pos, x_neg])
x_label = np.concatenate([x_label_pos, x_label_neg])
print(len(x_data)==len(x_label))
np.random.seed(seed)
np.random.shuffle(x_data)
np.random.seed(seed)
np.random.shuffle(x_label)
print(len(x_data)==len(x_label))
print(len(x_data))


y_data = np.concatenate([y_pos, y_neg])
y_label = np.concatenate([y_label_pos, y_label_neg])
print(len(y_data)==len(y_label))
np.random.seed(seed)
np.random.shuffle(y_data)
np.random.seed(seed)
np.random.shuffle(y_label)
print(len(y_data)==len(y_label))
print(len(y_data))



[ 1.  1.  1. ...,  1.  1.  1.]
[ 0.  0.  0. ...,  0.  0.  0.]
True
True
25000
True
True
25000


In [78]:
import bucket_io


ImportError: No module named 'bucket_io'

In [77]:
a = x_pos
b = x_neg
a = [[1,2,3],[4,5,6]]
al = np.ones(len(a))
b = [[7,8,9], [10,11,12]]
bl = np.zeros(len(b))
c = np.concatenate([a,b])
cl = np.concatenate([al, bl])
print(c)
print(cl)
np.random.seed(13)
np.random.shuffle(c)
np.random.seed(13)
np.random.shuffle(cl)
print(c)
print(cl)


[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
[ 1.  1.  0.  0.]
[[ 4  5  6]
 [10 11 12]
 [ 1  2  3]
 [ 7  8  9]]
[ 1.  0.  1.  0.]


In [64]:



for item in a:
    a_cnt = (np.bincount(item))
    # print('{}; {}; {}; {}; {}'.format(a_cnt, len(item), len(a_cnt), a_cnt.mean(), a_cnt.var()))
k = [95, 23, 123, 95, 23, 456, 222, 95, 123, 95, 123, 657]
#print(np.bincount(k)


x = a[100]
y = np.bincount(x)
ii = np.nonzero(y)[0]    
zip(ii,y[ii]) 
print(np.vstack((ii,y[ii])).T)
print(dic[80083])

IndexError: list index out of range

In [65]:
def create_bag_of_words(input_para):
    output_para = []
    
    
    return output_para

In [70]:
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [38]:
d = StringIO("M 21 72\nF 35 58")
np.loadtxt(d, dtype={'names': ('gender', 'age', 'weight'),'formats': ('S1', 'i4', 'f4')})

NameError: name 'StringIO' is not defined

In [17]:
from keras.datasets import imdb

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Using TensorFlow backend.


Loading data...
25000 train sequences
25000 test sequences


In [19]:
import lstm_bucketing



    train_sent, vocab = tokenize_text("./data/ptb.train.txt", start_label=start_label,
                                      invalid_label=invalid_label)
    val_sent, _ = tokenize_text("./data/ptb.test.txt", vocab=vocab, start_label=start_label,
                                invalid_label=invalid_label)

    data_train  = mx.rnn.BucketSentenceIter(train_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label)
    data_val    = mx.rnn.BucketSentenceIter(val_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label)


ImportError: No module named 'lstm_bucketing'